In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1725823912936_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [3]:
spark.sql("show databases").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|     namespace|
+--------------+
|       default|
| myspectrum_db|
|rawtotrusteddb|
+--------------+

In [4]:
# Set the database context
spark.sql("USE rawtotrusteddb")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [5]:
# Run a query on a table in the selected database
spark.sql("SHOW TABLES").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------------+-----------+
|     namespace|           tableName|isTemporary|
+--------------+--------------------+-----------+
|rawtotrusteddb|climate_change_in...|      false|
|rawtotrusteddb|climate_change_in...|      false|
|rawtotrusteddb|climate_change_vi...|      false|
|rawtotrusteddb|       co2_emissions|      false|
|rawtotrusteddb|co2_emissions_tru...|      false|
|rawtotrusteddb|    sea_level_change|      false|
|rawtotrusteddb|sea_level_change_...|      false|
+--------------+--------------------+-----------+

In [20]:
spark.sql("SELECT * FROM  sea_level_change_trusted").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|                year|siro_-_adjusted_sea_level__inches_#0|csiro_-_lower_error_bound__inches_#1|csiro_-_upper_error_bound__inches_#2|noaa_-_adjusted_sea_level__inches_#3|
+--------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+
|Figure 1. Global ...|                          Unnamed: 1|                          Unnamed: 2|                          Unnamed: 3|                          Unnamed: 4|
|Source: EPA's Cli...|                                    |                                    |                                    |                                    |
|Data source: CSIR...|                                    |                                    |                                    |            

In [25]:
from pyspark.sql.functions import col, when, regexp_replace

# Define the SQL query with escaped column names
sql_query = """
SELECT 
    CAST(year AS INT) AS Year,
    CAST(NULLIF(regexp_replace(`siro_-_adjusted_sea_level__inches_#0`, '[^0-9.-]', ''), '') AS DOUBLE) AS CSIRO_Adjusted_Sea_Level,
    CAST(NULLIF(regexp_replace(`noaa_-_adjusted_sea_level__inches_#3`, '[^0-9.-]', ''), '') AS DOUBLE) AS NOAA_Adjusted_Sea_Level,
    CAST(NULLIF(regexp_replace(`noaa_-_adjusted_sea_level__inches_#3`, '[^0-9.-]', ''), '') AS DOUBLE) - 
    CAST(NULLIF(regexp_replace(`siro_-_adjusted_sea_level__inches_#0`, '[^0-9.-]', ''), '') AS DOUBLE) AS Sea_Level_Difference
FROM 
    rawtotrusteddb.sea_level_change_trusted
WHERE 
    CAST(year AS INT) BETWEEN 1880 AND 2013
    AND `siro_-_adjusted_sea_level__inches_#0` != ''
    AND `noaa_-_adjusted_sea_level__inches_#3` != ''
ORDER BY 
    Year ASC
"""

# Execute the query and show results
result = spark.sql(sql_query)
result.show(truncate=False)

# Optional: If you want to perform further operations using DataFrame API
df = spark.table("rawtotrusteddb.sea_level_change_trusted")
df_filtered = df.filter(
    (col("year").cast("int").between(1880, 2013)) &
    (col("`siro_-_adjusted_sea_level__inches_#0`") != '') &
    (col("`noaa_-_adjusted_sea_level__inches_#3`") != '')
).select(
    col("year").cast("int").alias("Year"),
    when(col("`siro_-_adjusted_sea_level__inches_#0`") != '', 
         regexp_replace(col("`siro_-_adjusted_sea_level__inches_#0`"), "[^0-9.-]", "").cast("double"))
        .alias("CSIRO_Adjusted_Sea_Level"),
    when(col("`noaa_-_adjusted_sea_level__inches_#3`") != '', 
         regexp_replace(col("`noaa_-_adjusted_sea_level__inches_#3`"), "[^0-9.-]", "").cast("double"))
        .alias("NOAA_Adjusted_Sea_Level")
).withColumn(
    "Sea_Level_Difference",
    col("NOAA_Adjusted_Sea_Level") - col("CSIRO_Adjusted_Sea_Level")
).orderBy("Year")

df_filtered.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------------+-----------------------+--------------------+
|Year|CSIRO_Adjusted_Sea_Level|NOAA_Adjusted_Sea_Level|Sea_Level_Difference|
+----+------------------------+-----------------------+--------------------+
|1993|6.291338576             |6.297493046            |0.006154469999999357|
|1994|6.499999993             |6.310933553            |-0.1890664400000004 |
|1995|6.618110229             |6.452568661            |-0.16554156800000008|
|1996|6.787401568             |6.62763131             |-0.159770258        |
|1997|7.066929127             |6.733920367            |-0.33300876000000024|
|1998|6.665354324             |6.865806069            |0.20045174499999963 |
|1999|7.011811016             |6.848689771            |-0.16312124500000014|
|2000|7.062992119             |6.920582014            |-0.14241010499999973|
|2001|7.287401567             |7.087460011            |-0.19994155599999974|
|2002|7.381889756             |7.319697525            |-0.06219223100000004|

In [23]:
sql_query = """
SELECT 
    t1.country AS country,
    t1.iso2 AS iso_code_2,
    t1.iso3 AS iso_code_3,
    t1.f2003 AS temperature_change_2003, 
    t2.value AS co2_emissions_2003
FROM 
    rawtotrusteddb.climate_change_indicators_trusted t1
JOIN 
    rawtotrusteddb.co2_emissions_trusted t2
    ON t1.country = t2.`country_or_area#0`
WHERE 
    t2.year = 2003
"""

result = spark.sql(sql_query)
result.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+----------+----------+-----------------------+------------------+
|country               |iso_code_2|iso_code_3|temperature_change_2003|co2_emissions_2003|
+----------------------+----------+----------+-----------------------+------------------+
|Angola                |AO        |AGO       |0.889                  |0.924988063014922 |
|Albania               |AL        |ALB       |0.97                   |1.3611587779509   |
|United Arab Emirates  |AE        |ARE       |1.052                  |27.2248726413375  |
|Argentina             |AR        |ARG       |0.635                  |3.33489087930132  |
|American Samoa        |AS        |ASM       |0.716                  |null              |
|Antigua and Barbuda   |AG        |ATG       |0.783                  |4.6199167467179   |
|Australia             |AU        |AUS       |0.835                  |17.8945391341105  |
|Austria               |AT        |AUT       |1.464                  |9.16931675643542  |
|Burundi  